In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from io import BytesIO
from urllib.parse import urljoin, urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from PIL import Image
import os
import random
import re
import pandas as pd
import hashlib
from datetime import datetime
import urllib.parse
from json import dumps, loads
from shutil import copy2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.chdir('/content/drive/MyDrive/research-similarity/Scraping')

In [ ]:
url = 'https://english.elpais.com/health/'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'lxml')

In [ ]:
main_content = soup.find('div', class_='b-b b-au_b')

for article_card in main_content.find_all('article'):
    print(article_card.find('a')['href'])

https://english.elpais.com/health/2024-09-14/mpox-an-emergency-for-children.html
https://english.elpais.com/health/2024-09-13/being-distracted-doesnt-mean-you-have-adhd-the-danger-of-self-diagnosis-in-teenagers.html
https://english.elpais.com/health/2024-09-13/diego-hidalgo-banning-the-sale-of-phones-to-teenagers-would-empower-parents.html
https://english.elpais.com/health/2024-09-12/extreme-weather-and-epidemics-how-75-people-were-hospitalized-after-eating-contaminated-lettuce.html
https://english.elpais.com/science-tech/2024-09-11/new-generation-of-radioactive-drugs-strike-cancer-with-molecular-precision.html
https://english.elpais.com/health/2024-09-10/the-science-of-stoicism-does-it-really-improve-mental-health.html
https://english.elpais.com/science-tech/2024-09-10/pathogenic-microbes-can-travel-thousands-of-miles-through-the-atmosphere.html
https://english.elpais.com/health/2024-09-06/biological-samples-from-flight-attendants-may-shed-light-on-how-jet-lag-affects-health.html
http

In [ ]:
soup.find('div', class_='b-au_o').find('article').find('a')['href']

'https://english.elpais.com/health/2024-09-16/immunotherapy-achieves-remarkable-survival-rates-in-a-handful-of-tumors.html'

In [ ]:
url = 'https://english.elpais.com/economy-and-business/2024-08-27/ukraine-and-gaza-wars-boost-the-value-of-major-arms-manufacturers.html'
url1 = 'https://english.elpais.com/health/2024-09-13/being-distracted-doesnt-mean-you-have-adhd-the-danger-of-self-diagnosis-in-teenagers.html'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'lxml')

In [ ]:
headline = soup.find('h1').text
datetime_str = soup.find('div', class_='a_md_f').find('a').get('data-date')[:-6]
time = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S")

In [ ]:
article_content = soup.find('div', class_='a_c clearfix')

for img in article_content.find_all('img'):
    print(img)

<img alt="Six F-16s from the Polish Air Force, on August 15 in Warsaw." class="_re a_m-h" decoding="auto" height="276" loading="eager" sizes="(min-width: 1199px) 760px,(min-width: 1001px) cal(100vw - 62vw),(min-width: 768px) 767px, 100vw" src="https://imagenes.elpais.com/resizer/v2/PNN5ARAXF2DW6DU2PK76QSLFFE.jpg?auth=ffa6f8012a1bbdc2275f59cddbaaef95b69f8d98027406a69efeabc125068c91&amp;width=414" srcset="https://imagenes.elpais.com/resizer/v2/PNN5ARAXF2DW6DU2PK76QSLFFE.jpg?auth=ffa6f8012a1bbdc2275f59cddbaaef95b69f8d98027406a69efeabc125068c91&amp;width=414 414w,https://imagenes.elpais.com/resizer/v2/PNN5ARAXF2DW6DU2PK76QSLFFE.jpg?auth=ffa6f8012a1bbdc2275f59cddbaaef95b69f8d98027406a69efeabc125068c91&amp;width=828 640w,https://imagenes.elpais.com/resizer/v2/PNN5ARAXF2DW6DU2PK76QSLFFE.jpg?auth=ffa6f8012a1bbdc2275f59cddbaaef95b69f8d98027406a69efeabc125068c91&amp;width=980 1000w,https://imagenes.elpais.com/resizer/v2/PNN5ARAXF2DW6DU2PK76QSLFFE.jpg?auth=ffa6f8012a1bbdc2275f59cddbaaef95b69f8d98

In [ ]:
soup.find('header', class_='a_e _g _g-xs').find('img')

In [5]:
def create_directories(base_url, categories, label='images'):
    # create the following dir struct; outputs > base website > categories
    base_dir = os.path.join('output', label, urlparse(base_url).netloc)
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    for category, _ in categories:
        category_dir = os.path.join(base_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

    return base_dir

In [6]:
def get_articles_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    main_content = soup.find('div', class_='b-b b-au_b')
    articles_links = [soup.find('div', class_='b-au_o').find('article').find('a')['href']]

    for article_card in main_content.find_all('article'):
        link = article_card.find('a')['href']
        if link not in articles_links:
            articles_links.append(link)

    return articles_links

def article_scrapper(url):
    articles_links = get_articles_links(url)
    data = []

    def link_fix(link):
        index = link.find('&')
        if index != -1:
            return link[:index]
        else:
            return link

    def helper_scrapper(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'lxml')
        try:
            headline = soup.find('h1').text
            datetime_str = soup.find('div', class_='a_md_f').find('a').get('data-date')[:-6]
            time = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S")
            article_content = soup.find('div', class_='a_c clearfix')
            images = []
            main_image = soup.find('header', class_='a_e _g _g-xs').find('img')
            if (main_image):
                images.append((main_image.get('alt'), link_fix(main_image.get('src'))))

            for img in article_content.find_all('img'):
                img_alt = img.get('alt', '')  # Safely get the 'alt' attribute
                img_src = link_fix(img.get('src', ''))  # Safely get the 'src' attribute

                if img_src and (img_alt, img_src) not in images:
                    images.append((img_alt, img_src))

            return headline, time, images
        except:
            return None

    for url in articles_links:
        result = helper_scrapper(url)
        if result is not None:
            data.append(result)

    return data

In [7]:
def get_latest_articles(data, n=10):
    seen_headlines = set()
    unique_data = []

    for record in data:
        headline = record[0]
        if headline not in seen_headlines:
            seen_headlines.add(headline)
            unique_data.append(record)

    return sorted(unique_data, key=lambda x: x[1], reverse=True)[:n]

def download_image(img_url, save_dir, img_name):
    try:
        if not img_url.startswith('data:'):
            response = requests.get(img_url)
            img_data = response.content
            img = Image.open(BytesIO(img_data))
            width, height = img.size

            # Only save images larger than 100x100 pixels
            if width >= 100 and height >= 100:
                with open(os.path.join(save_dir, img_name), 'wb') as img_file:
                    img_file.write(img_data)
    except:
        pass

def download_images(category_url, save_dir, data):

    with open(os.path.join(save_dir, 'labels.csv'), 'w') as f:
        f.write('image number,alt,article_heading\n')

    records = []

    # parallising the downloads to make it faster
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        headlines = []
        for x, tuple in enumerate(data):
            headline, _, images_list = tuple
            for i, img in enumerate(images_list):
                alt_txt, img_url = img
                if img_url and not img_url.startswith('data:'):
                    img_url = urljoin(category_url, img_url)
                    combined_str = f"{alt_txt}{headline}".encode()
                    img_name = f'image_{x+1}_{i+1}.jpg'
                    records.append(f'{img_name},{alt_txt.replace(",", "")},{headline.replace(",", "")}\n')
                    futures.append(executor.submit(download_image, img_url, save_dir, img_name))

        with open(os.path.join(save_dir, 'labels.csv'), 'a') as f:
            f.writelines(records)

        for future in as_completed(futures):
            future.result()


In [8]:
categories = [
    ("Health", "https://english.elpais.com/health"),
    ("Economy and Business", "https://english.elpais.com/economy-and-business/"),
    ("U.S.", "https://english.elpais.com/usa/"),
    ("Technology", "https://english.elpais.com/technology/"),
    ("Climate", "https://english.elpais.com/climate/"),
    ("People", "https://english.elpais.com/people/"),
    ("Lifestyle", "https://english.elpais.com/lifestyle/"),
    ("Opinion", "https://english.elpais.com/opinion/"),
    ("Culture", "https://english.elpais.com/culture/"),
    ("Sports", "https://english.elpais.com/sports/"),
    ("EPS", "https://english.elpais.com/eps/")
]

base_url = 'https://english.elpais.com/'

base_dir = create_directories(base_url, categories)

In [9]:
for category, category_url in tqdm(categories, desc='Downloading images for every category'):
    try:
        category_dir = os.path.join(base_dir, category)
        data = article_scrapper(category_url)
        download_images(category_url, category_dir, get_latest_articles(data))
    except:
        print(category)
        continue

In [19]:
def create_file(base_dir):
    for category in os.listdir(base_dir):
        try:
            df = pd.read_csv(f'{base_dir}{category}/labels.csv')
            n, _ = df.shape

            pairs = []

            for i in range(n):
                for j in range(i + 1, n):
                    article_1 = int(re.search(r'\d+(?=_|$)',list(df.iloc[i])[0]).group())
                    article_2 = int(re.search(r'\d+(?=_|$)',list(df.iloc[j])[0]).group())
                    if article_1 != article_2:
                        pairs.append((list(df.iloc[i]), list(df.iloc[j])))

            with open(f'{base_dir.split("/")[2]}_pairs_{category}.csv', 'w') as f:
                for i, pair in enumerate(pairs):
                    p1, p2 = pair
                    img1, alt1, headline1 = p1
                    img2, alt2, headline2 = p2
                    f.write(f'{i+1},{headline1},{headline2}\n')
                    f.write(f',https://raw.githubusercontent.com/ayainfida/news-scrapper/main/output/images/english.elpais.com/{urllib.parse.quote(category)}/{img1},https://raw.githubusercontent.com/ayainfida/news-scrapper/main/output/images/english.elpais.com/{urllib.parse.quote(category)}/{img2}\n')
                    f.write(f',{alt1},{alt2}\n')

        except:
            continue

In [18]:
for category in os.listdir('output/images/english.elpais.com/'):
    df = pd.read_csv(f'output/images/english.elpais.com/{category}/labels.csv')
    n, _ = df.shape

    if n + 1 != len(os.listdir(f'output/images/english.elpais.com/{category}')):
        print(category,n+1, len(os.listdir(f'output/images/english.elpais.com/{category}')))

In [20]:
create_file('output/images/english.elpais.com/')

In [21]:
num_pairs = []

for file in os.listdir():
    if file.startswith('english.elpais.com_pairs'):
        try:
            df = pd.read_csv(file)
            num_pairs.append((int(list(df.iloc[-3])[0]), file))
        except:
            continue

In [22]:
sorted_array = sorted(num_pairs, key=lambda x: x[0])
sorted_array

[(125, 'english.elpais.com_pairs_Economy and Business.csv'),
 (174, 'english.elpais.com_pairs_U.S..csv'),
 (174, 'english.elpais.com_pairs_Technology.csv'),
 (183, 'english.elpais.com_pairs_Lifestyle.csv'),
 (226, 'english.elpais.com_pairs_Health.csv'),
 (392, 'english.elpais.com_pairs_Sports.csv'),
 (395, 'english.elpais.com_pairs_People.csv'),
 (425, 'english.elpais.com_pairs_Culture.csv'),
 (430, 'english.elpais.com_pairs_Climate.csv'),
 (688, 'english.elpais.com_pairs_EPS.csv')]